# APENDIX: Ridge Closed-Form vs. Gradient Descent in Regularized Linear Regression

In this section we compare two ways of training **regularized linear regression**:

1. **Gradient descent** (iterative optimization).
2. **Ridge closed-form solution** (analytical solution).

Both aim to minimize the *same* regularized cost function, but they behave quite differently in practice, especially when we use **high-degree polynomial features** (e.g., degree 10) as in our BootCamp example.

We will:

- Explain why gradient descent sometimes misbehaves in this setting.
- Motivate why the ridge closed-form solution is a good **alternative algorithm** for this specific demo.
- Clarify when to prefer each approach.

---

## 1. Problem Setting

We consider **regularized linear regression** with parameters $\vec{w} \in \mathbb{R}^n$ and bias $b \in \mathbb{R}$.

For a feature vector $\vec{x}^{(i)} \in \mathbb{R}^n$, the hypothesis is

$$
f_{\vec{w}, b}^{(i)}(\vec{x}^{(i)}) = f_{\vec{w}, b}(\vec{x}^{(i)}) 
= \vec{w}\cdot\vec{x}^{(i)} + b.
$$

The **regularized cost function** is

$$
J_{\text{reg}}(\vec{w}, b)
=
\frac{1}{2m}
\sum_{i=1}^m
\left(
f_{\vec{w}, b}^{(i)}(\vec{x}^{(i)}) - y^{(i)}
\right)^2
+
\frac{\lambda}{2m}
\sum_{j=1}^n w_j^2,
$$

where:

- $m$: number of training examples.
- $\lambda \ge 0$: regularization strength.
- The sum over $j$ includes only the components of $\vec{w}$ (we do **not** regularize $b$).

We often obtain features via **polynomial expansion**, for example in 1D:

$$
\vec{x}^{(i)} = 
\big[x^{(i)}, (x^{(i)})^2, \dots, (x^{(i)})^{10}\big]
$$

for degree-10 polynomial regression.

---

## 2. Gradient Descent: General but Sensitive

### 2.1 Gradient Descent Updates

We first derive the derivatives used in **gradient descent**.

Define the error for each example:

$$
e^{(i)} = f_{\vec{w}, b}^{(i)}(\vec{x}^{(i)}) - y^{(i)}.
$$

Then the gradients of the regularized cost are:

For each weight $w_j$,

$$
\frac{\partial J_{\text{reg}}}{\partial w_j}
=
\frac{1}{m}
\sum_{i=1}^m e^{(i)} x_j^{(i)}
+
\frac{\lambda}{m} w_j,
$$

and for the bias,

$$
\frac{\partial J_{\text{reg}}}{\partial b}
=
\frac{1}{m}
\sum_{i=1}^m e^{(i)}.
$$

Gradient descent updates the parameters as:

$$
w_j := w_j - \alpha \frac{\partial J_{\text{reg}}}{\partial w_j},
\quad
b := b - \alpha \frac{\partial J_{\text{reg}}}{\partial b},
$$

where $\alpha > 0$ is the **learning rate**.

### 2.2 Why Gradient Descent Is Important

Gradient descent is central in machine learning because:

- It works for **any differentiable** cost function.
- It scales to very **large datasets**.
- It is the foundation for training more complex models:
  - Logistic regression,
  - Neural networks,
  - Deep learning architectures.

In our BootCamp, we use gradient descent to:

- Make students compute and understand **derivatives**,
- Show how the model is updated **iteratively**,
- Connect the math with actual **code**.

### 2.3 Problems with High-Degree Polynomial Features

However, when we use **high-degree polynomials**, gradient descent can become **numerically unstable**:

1. **Large feature values**

   If $x \in [-3, 3]$, then

   $$
   x^{10} \approx 3^{10} = 59\,049,
   $$

   so some components of $\vec{x}^{(i)}$ are huge.

   This leads to:

   - Very large predictions $f_{\vec{w}, b}^{(i)}(\vec{x}^{(i)})$,
   - Very large errors $e^{(i)}$,
   - Very large squared errors $(e^{(i)})^2$,
   - Overflow warnings like `RuntimeWarning: overflow encountered in square`.

2. **Exploding gradients**

   Because the cost and errors are large, gradients also become large:

   - Parameter updates become huge,
   - The algorithm “jumps” instead of gradually converging,
   - The cost can diverge instead of decreasing.

3. **Sensitivity to hyperparameters**

   To stabilize gradient descent we often need to:

   - Carefully **scale features**,
   - Use very small learning rates $\alpha$,
   - Increase the number of iterations,
   - Tune $\alpha$ by trial and error.

   Even then, a slight misconfiguration can result in:

   - Very noisy or wrong curves,
   - Poor visualization of overfitting vs regularization.

### 2.4 Consequence for the Classroom Demo

For the **conceptual demo** we want:

- Degree 10, $\lambda = 0$: **strong overfit**, very wiggly curve.
- Degree 10, small $\lambda$: **smooth but flexible**, good fit.
- Degree 10, large $\lambda$: **very smooth**, underfit almost like a line.

Gradient descent with high-degree polynomial features:

- May fail to converge,
- May produce all three curves looking similar and underfit,
- Or even produce numerical errors and NaNs.

This makes it hard to show students the **clean, expected behavior** of regularization.

---

## 3. Ridge Closed-Form: Exact and Stable Alternative

For **regularized linear regression**, there is a **closed-form analytical solution**, known as **ridge regression**.

### 3.1 Ridge Closed-Form Solution

Let $X \in \mathbb{R}^{m \times n}$ be the design matrix of features (without bias), and $y \in \mathbb{R}^m$ the vector of targets.

We construct $X_{\text{bias}} \in \mathbb{R}^{m \times (n+1)}$ by adding a column of ones for the bias:

$$
X_{\text{bias}} = [X \;|\; \mathbf{1}],
$$

and define

$$
\theta =
\begin{bmatrix}
\vec{w} \\
b
\end{bmatrix}
\in \mathbb{R}^{n+1}.
$$

Then the **ridge solution** is

$$
\theta
=
\left(
X_{\text{bias}}^\top X_{\text{bias}} + \lambda I_{\text{reg}}
\right)^{-1}
X_{\text{bias}}^\top y,
$$

where $I_{\text{reg}}$ is the identity matrix of size $(n+1) \times (n+1)$ but with the last diagonal entry set to 0, so we **do not regularize the bias**:

- For $j = 1, \dots, n$: we regularize $w_j$,
- For the last entry (bias $b$): no regularization.

This gives us $\vec{w}$ and $b$ **in one shot**, without iterations.

### 3.2 Advantages of the Ridge Closed-Form Solution

Compared to gradient descent, the ridge closed-form solution:

- Is **exact** (for this cost function): we obtain the global minimizer directly.
- Has **no learning rate** to tune.
- Is **numerically stable** for polynomial features of moderate degree.
- Produces **predictable, clean plots**:
  - $\lambda = 0$: strong overfitting (wiggly curve),
  - Intermediate $\lambda$: controlled complexity,
  - Large $\lambda$: underfitting (almost linear).

For our polynomial regression demo, this is ideal:  
students can clearly see the effect of **the same degree, different $\lambda$** on the shape of the curve.

### 3.3 Limitations of the Closed-Form Solution

The ridge closed-form method also has limitations:

- It requires computing the inverse (or pseudo-inverse) of a $(n+1) \times (n+1)$ matrix.
  - This is fine for small or moderate $n$,
  - But can be expensive when $n$ is very large.
- It only exists for **linear models** with quadratic costs.
  - There is no closed-form solution for logistic regression,
  - Nor for neural networks and most deep learning models.

So, while perfect for our **regularized linear regression demo**, it is **not a general replacement** for gradient descent.

---

## 4. Why Use an Alternative Algorithm (Ridge) to Gradient Descent Here?

Putting it all together:

- For teaching **derivatives** and **algorithmic thinking**, we use:
  - Gradient descent,
  - Explicit expressions for $\frac{\partial J_{\text{reg}}}{\partial w_j}$ and $\frac{\partial J_{\text{reg}}}{\partial b}$,
  - Simple models (low-degree or well-scaled features).

- For teaching the **effect of regularization** on a **high-degree polynomial model**, we prefer:
  - The ridge closed-form solution,
  - Because it is more **robust**, **stable**, and **does not require tuning**,
  - And it gives the **expected visual behavior** (overfit vs underfit) in a clean way.

In short:

- **Gradient descent** is the **general-purpose learning algorithm** you will use everywhere.
- **Ridge closed-form** is a **specialized, more reliable tool** for this particular kind of model, perfect for showing how $\lambda$ controls model complexity.

---

## 5. Takeaways for Students

1. Gradient descent and ridge closed-form are two ways of solving the **same optimization problem** for regularized linear regression.
2. Gradient descent:
   - Is **general** and used in many models,
   - But can be **numerically fragile** in high-degree polynomial settings.
3. Ridge closed-form:
   - Is **exact** and **stable** for this specific case,
   - Makes the effect of **regularization** visually clear and reliable.
4. In real practice:
   - Use closed-form when the model is simple and the feature dimension is moderate,
   - Use gradient descent (or variants like SGD, Adam) for large-scale or more complex models.

This is why, in our BootCamp materials, we keep **gradient descent** to teach the learning algorithm, but we use the **ridge closed-form** for the key **visualization of regularization** in high-degree polynomial regression.